In [1]:
import pandas as pd
import math
import requests
from bs4 import BeautifulSoup
import json
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor
import re
import csv
import time

In [2]:
df = pd.read_csv('Data/yad_2_data.csv')
def algo_accuracy(df):
    df['Accuracy'] = 0
    
    for row in range(df.shape[0]):
        accuracy = 20
        
        if pd.notna(df.at[row, 'Neighborhood']):
            accuracy += 10
            
        if pd.notna(df.at[row, 'Home_number']):
            accuracy += 10
            
        if df.at[row, 'Long'] != 0:
            accuracy += 10
            
        if df.at[row, 'Lat'] != 0:
            accuracy += 10
            
        if df.at[row, 'Size'] != 0 and pd.notna(df.at[row, 'Size']):
            accuracy += 10
            
        if df.at[row, 'Rooms'] != 0 and pd.notna(df.at[row, 'Rooms']):
            accuracy += 10
        
        if pd.notna(df.at[row, 'Floor']):
            accuracy += 10
            
        if pd.notna(df.at[row, 'Floors']):
            accuracy += 10
        
        df.at[row, 'Accuracy'] = accuracy
        
    return df

def drop_rows_with_keyword(df):
    keywords = ['פנטאהוס','בדמי מפתח', 'דמי מפתח', 'משרד', 'לא למגורים','סכסוך',
                'משרדים' ,'אולם','מפתח' ,'לשימור','דרוש שיפוץ','דרושה שיפוץ','להריסה','דירת פאר','מחולקת']
    df['Text'] = df['Text'].astype(str)

    # Generate the conditions for dropping rows
    conditions = [df['Text'].str.contains(keyword, case=False) for keyword in keywords]
    drop_mask = pd.concat(conditions, axis=1).any(axis=1)

    # Drop the rows that meet any of the conditions
    df = df[~drop_mask]

    return df

df = algo_accuracy(df)
df = drop_rows_with_keyword(df) 
try:
    df = df.drop(['level_0', 'index'], axis=1)
except:
    pass
df.to_csv('Data/yad_2_data.csv' ,index=False )
print(df.shape)

(5714, 39)


In [3]:
def get_gush_chelka_api(address):
    headers = {
        'sec-ch-ua': '"Not/A)Brand";v="99", "Google Chrome";v="115", "Chromium";v="115"',
        'auth_data': '{"api_token":"03387185-da2d-4cb4-a3c6-2292a3df7915","user_token":"","domain":"www.gov.il","token":"qMZlz1IwqfKc6WhO6qet3OHGjLusa/tBvtVdRBurehytCsrtnVhcI81jhg/2pDc39ChRLpK7C+mzq4rg5GjqdHfTQJANTYMPAmVWrfMwL4O2AMtdLCq+QEt/inww791B+vWYq7Z0PiAq8lUPSBuB1//YGsM/BvB9daJoQwguhya+2NxIm2JK2gfMp+RnSHzGdn4LFWLuLw3lHHYzhGT8Q8+3vzRbF/vSl79P7qNAoLxGPApgGa80OA==","user_id":178592,"isAdmin":false,"expires":"2023-08-15T21:20:19.0844919+03:00"}',
        'sec-ch-ua-mobile': '?0',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36',
        'Content-Type': 'application/json; charset=UTF-8',
        'Accept': 'application/json, text/javascript, */*; q=0.01',
        'Referer': 'https://www.gov.il/',
        'sec-ch-ua-platform': '"Windows"',
    }

    json_data = {
        'type': 0,
        'address': address,
    }

    response = requests.post('https://ags.govmap.gov.il/Api/Controllers/GovmapApi/SearchAndLocate', headers=headers, json=json_data)
    if response.status_code == 200:
        data = response.json()
        if data['status'] != 0:
            return False
        if 'data' in data and len(data['data']) > 0 and 'Values' in data['data'][0]:
            return [int(value) for value in data['data'][0]['Values']]
    return False

In [4]:
def add_gush_helka_tat(df, city='תל אביב'):
    df.loc[:, 'Gush'] = np.nan
    df.loc[:, 'Helka'] = np.nan
    df.loc[:, 'Tat'] = np.nan

    df1_gov = pd.read_csv("Data/Real_Estate_TLV_GOVMAPS_1.csv", index_col=0)
    df2_gov = pd.read_csv("Data/Real_Estate_TLV_GOVMAPS_2.csv", index_col=0)
    df_gov = pd.merge(df1_gov, df2_gov, how='outer')
    df_gov[['Gush', 'Helka', 'Tat']] = df_gov['GUSHHELKATAT'].str.split('-|/', n=2, expand=True).astype(np.int32)
    df_gov['Home_number'] = df_gov['ADDRESS'].str.extract('(\d+)').astype(np.int32)
    
    # Level 0: Government Maps API
    for index, row in df.iterrows():
        if pd.notna(row['Home_number']):
            home_number = int(row['Home_number'])
            street = row['Street']
            
            for home_n in range(0,10,2):
                home_number = home_number + home_n
                address = str(street)+' '+str(home_number)+' '+city
                gush_chelka = get_gush_chelka_api(address)
                if gush_chelka:
                    df.loc[index, 'Gush'] = gush_chelka[0]
                    df.loc[index, 'Helka'] = gush_chelka[1]
                    break
            
    # Level 1: Government Maps
    for index, row in df.iterrows():
        if pd.notna(row['Home_number']) and pd.isna(row['Gush']):
            home_number = row['Home_number']
            street = row['Street']

            matches = df_gov.loc[
                (df_gov['STREENNAME'].str.contains(street)) & (df_gov['Home_number'] == home_number), ['Gush', 'Helka', 'Tat']]

            if not matches.empty:
                df.loc[index, 'Gush'] = matches['Gush'].values[0]
                df.loc[index, 'Helka'] = matches['Helka'].values[0]
                df.loc[index, 'Tat'] = matches['Tat'].values[0]

    # Level 2: Nadlan Database
    print(df['Gush'].notna().sum())
    df_nadlan = pd.read_csv("Data/Nadlan.csv")
    df_nadlan[['Gush', 'Helka', 'Tat']] = df_nadlan['GUSH'].str.split('-|/', n=2, expand=True).astype(np.int32)
    df_nadlan = df_nadlan.drop(columns='GUSH', axis=1)
    df_nadlan[['Street', 'Home_number']] = df_nadlan['DISPLAYADRESS'].str.extract('(.+)\s+(\d+)')
    df_nadlan.dropna(subset=['Home_number'], inplace=True)
    df_nadlan['Home_number'] = df_nadlan['Home_number'].astype(np.int32)

    for index, row in df.iterrows():
        if pd.notna(row['Home_number']) and pd.isna(row['Gush']):
            home_number = row['Home_number']
            street = row['Street']

            matches = df_nadlan.loc[
                (df_nadlan['Street'].str.contains(street)) & (df_nadlan['Home_number'] == home_number), ['Gush', 'Helka', 'Tat']]

            if not matches.empty:
                df.loc[index, 'Gush'] = matches['Gush'].values[0]
                df.loc[index, 'Helka'] = matches['Helka'].values[0]
                df.loc[index, 'Tat'] = matches['Tat'].values[0]

    # Level 3: Addresses
    print(df['Gush'].notna().sum())
    df_address = pd.read_csv("Data/Addresses.csv")
    for index, row in df.iterrows():
        if pd.notna(row['Home_number']):
            home_number = row['Home_number']
            street = row['Street']

            gush = df_address.loc[(df_address['ms_bayit'] == home_number) & (df_address['t_rechov'].str.contains(street)), 'ms_gush']
            helka = df_address.loc[(df_address['ms_bayit'] == home_number) & (df_address['t_rechov'].str.contains(street)), 'ms_chelka']

            if not gush.empty:
                df.at[index, 'Gush'] = gush.iloc[0]
            if not helka.empty:
                df.at[index, 'Helka'] = helka.iloc[0]

            if gush.empty and helka.empty:
                street_parts = street.split(' ')
                last_part = street_parts[-1]
                matching_streets = df_address[df_address['t_rechov'].str.contains(last_part)]
                if not matching_streets.empty:
                    gush = matching_streets.loc[matching_streets['ms_bayit'] == home_number, 'ms_gush']
                    helka = matching_streets.loc[matching_streets['ms_bayit'] == home_number, 'ms_chelka']
                    if not gush.empty:
                        df.at[index, 'Gush'] = gush.iloc[0]
                    if not helka.empty:
                        df.at[index, 'Helka'] = helka.iloc[0]

    df = df.dropna(subset=['Gush', 'Helka'])
    df.loc[:, 'Gush'] = df['Gush'].astype(np.int32)
    df.loc[:, 'Helka'] = df['Helka'].astype(np.int32)

    print(df['Gush'].notna().sum())
    return df

df = add_gush_helka_tat(df)
print(f'shape {df.shape}')
# shape (1637, 42)


680
746
1553
shape (1553, 42)


C:\Users\yoavl\anaconda3\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [5]:
df = df.drop(columns=['Balconies','Parking','Balconies','Immediate'
                      ,'Ac','Furniture','On_pillars','Elevator','Storeroom','Shelter',
                     'Text','Date_of_entry','Images','Handicapped','Floors_text','Agency', 'Garden_size' 
                      , 'Date_added', 'City_code','City'], axis=1)


def update_asset_conditon(df):
    '''
    My index:
    1- new from builder
    2 - new 
    3 - good
    4 - after reinovtions 
    5 - need reinovtions
    '''
    df.rename(columns = {'Asset_classification':'New'}, inplace = True)
    df['New'] = df['New'].astype(np.int32)
    df.loc[df['New'] == 2, 'New'] = 4 
    df.loc[df['New'] == 6, 'New'] = 2
    
    # 1 new condtion else 0
    df['New'] = df['New'].apply(lambda x: 1 if x in [1, 2] else 0)
    
    return df
    
def update_neighborhood_street(df):
    df['Street'] = df['Street'].str.strip()
    old = "'"
    new = ''
    df['Street'] = df['Street'].str.replace(old,new)
    df['Neighborhood'] = df['Neighborhood'].str.replace(old,new)
    
    old = 'כרמייה'
    new = 'כרמיה'
    df['Street'] = df['Street'].str.replace(old,new)

    df['Neighborhood'] = df['Neighborhood'].str.strip()
    old = '-'
    new = ' '
    df['Neighborhood'] = df['Neighborhood'].str.replace(old,new)
    old = 'נווה אביבים'
    new = 'נווה אביבים וסביבתה'
    df['Neighborhood'] = df['Neighborhood'].str.replace(old,new)
    df['Neighborhood'] = df['Neighborhood'].str.replace('לב תל אביב, לב העיר צפון','הצפון הישן החלק הצפוני')
    df['Neighborhood'] = df['Neighborhood'].str.replace('נוה','נווה')
    df['Neighborhood'] = df['Neighborhood'].str.replace('גני צהלה, רמות צהלה','גני צהלה')
    df['Neighborhood'] = df['Neighborhood'].str.replace('נווה אליעזר וכפר שלם מזרח','כפר שלם מזרח נווה אליעזר')
    df['Neighborhood'] = df['Neighborhood'].str.replace('גני שרונה, קרית הממשלה','גני שרונה')
    old = 'הצפון הישן   דרום'
    new = 'הצפון החדש החלק הדרומי'
    df['Neighborhood'] = df['Neighborhood'].str.replace(old,new)
    df['Neighborhood'] = df['Neighborhood'].str.replace('מכללת תל אביב יפו, דקר','דקר')

    return df
 
    
df = update_asset_conditon(df)
df = update_neighborhood_street(df)

In [6]:
def convert_coordinates(lat_long_list):
    '''
    This function use to convert UTM coordinates to ITM coordinates using outside website with selenium 
    '''
    service = Service("C:/Users/yoavl/NextRoof/chromedriver_win32/chromedriver.exe")
    driver = webdriver.Chrome(service=service)
    driver.get("https://zvikabenhaim.appspot.com/software/ITM/")

    # locate all the input fields and button
    lat_input = driver.find_element(By.XPATH, '//*[@id="lat"]')
    lon_input = driver.find_element(By.XPATH, '//*[@id="long"]')
    convert_button = driver.find_element(By.XPATH, '//form[1]/table/tbody/tr[3]/td/input')
    east_span = driver.find_element(By.XPATH, '//*[@id="itm_east"]')
    north_span = driver.find_element(By.XPATH, '//*[@id="itm_north"]')
    
    
    coverted_list = []
    for cords in lat_long_list:
        east = cords[0]
        north = cords[1]
        if cords[0] != 0.0 and cords[1] != 0.0:
            lat_input.clear()
            lon_input.clear()

            # send lat and long to input fields

            lat_input.send_keys(cords[0])
            lon_input.send_keys(cords[1])

            convert_button.click()


            east = east_span.text
            north = north_span.text
            
            
        coverted_list.append((east,north))
        

    driver.quit()
    return coverted_list

def convert_lat_long(df):

    def get_long_lat_tuples(df):
        long_lat_tuples = []
        for index, row in df.iterrows():
            lat = row['Lat']
            long = row['Long']

            try:
                lat = float(lat)
            except:
                lat = 0.0
            try:
                long = float(long)
            except:
                long = 0.0  
            long_lat_tuples.append((lat, long))
        return long_lat_tuples
    
    new_cord = convert_coordinates(get_long_lat_tuples(df))
    
    for index, row in df.iterrows():  
        try:
            df.at[index,'Lat'] = new_cord[index][0]
            df.at[index,'Long'] = new_cord[index][1]
        except:
            pass
    return df

In [7]:
def find_avg_cords_by_street():
    df = pd.read_csv("Data/Nadlan_clean.csv",index_col=0)
    df = df.dropna(subset=['Long','Lat','Street']).reset_index(drop=True)
    df['Long'] = df['Long'].astype(int)
    df['Lat'] = df['Lat'].astype(int)
    
    # create unique streets list
    streets = df['Street'].unique()
    
    street_cords_avg = {}
    for street in streets:
        street_df = df[df['Street'] == street]
        
        lat_mean = street_df['Lat'].mean()
        long_mean = street_df['Long'].mean()

        street_cords_avg[street] = (long_mean ,lat_mean)
        
    return street_cords_avg
        
def complete_long_lat_columns(df,street_cords_avg):
    for index, row in df.iterrows():
        long = row['Long']
        lat = row['Lat']
        street = row['Street']
        
        try:
            cords = street_cords_avg[street]         
            if long == 0.0 or lat == 0.0:
                df.at[index, 'Long'] = cords[1]
                df.at[index, 'Lat'] = cords[0]      
        except:
            pass

    
    df.rename(columns = {'Lat':'Temp'}, inplace = True)
    df.rename(columns = {'Long':'Lat'}, inplace = True)
    df.rename(columns = {'Temp':'Long'}, inplace = True)

    df = df.loc[(df['Lat'] != 0.0) & (df['Long'] != 0.0)]
    
    df.loc[:, 'Long'] = df['Long'].astype(np.int32)
    df.loc[:, 'Lat'] = df['Lat'].astype(np.int32)
    
    return df
            
# df = convert_lat_long(df)
# df =complete_long_lat_columns(df,find_avg_cords_by_street())
print(df.shape)

(1553, 23)


In [8]:
df = df.dropna(subset=['Street']).reset_index(drop=True)
print(df.shape)
#1232 rows × 23 columns


# (2464, 18)

(1553, 23)


In [9]:
def add_neighborhood():
    df = pd.read_csv("Data/Real_Estate_TLV_GOVMAPS_1.csv",index_col=0)
    neighborhoods = {}
    for index, row in df.iterrows():
        neighborhood = row['NEIGHBORHOOD']
        street = row['STREENNAME']
        if neighborhood not in neighborhoods:
            neighborhoods[neighborhood] = set()
        neighborhoods[neighborhood].add(street)
    return neighborhoods

def complete_neighborhood_column(df, neighborhoods):
    # Loop through each row in the DataFrame
    for index, row in df.iterrows():
        street = row['Street']
        
        # Loop through each neighborhood in the neighborhoods dictionary
        for neighborhood, streets in neighborhoods.items():
            # Check if the street is in the set of streets for the neighborhood
            if street in streets:
                # If it is, assign the neighborhood to the 'neighborhood' column for this row
                df.at[index, 'Neighborhood'] = neighborhood
                break # Break out of the inner loop once we find a match
    
    return df

df = complete_neighborhood_column(df,add_neighborhood())

In [10]:
def create_building_year_list():
    df = pd.read_csv("Data/Nadlan_clean.csv",index_col=0)
    df = df.dropna(subset=['Floors','Street','Build_year']).reset_index(drop=True)
    
    add_build_year = []
    df['Floors'] = df['Floors'].astype(int)
    df['Street'] = df['Street'].astype(str)
    df['Build_year'] = df['Build_year'].astype(int)
    
    for index, row in df.iterrows():
        floors = row['Floors']
        street = row['Street']
        build_year = row['Build_year']
        
        build_string = (str(floors)+str(street)+str(build_year)).strip()
        
        add_build_year.append(build_string)
    return list(set(add_build_year))

def complete_build_year_column(df, add_build_year):
    df['Build_year'] = 0
    df['Floors'] = df['Floors'].fillna(0)
    
    df['Floors'] = df['Floors'].astype(int)
    df['Street'] = df['Street'].astype(str)
    
    # Loop through each row in the DataFrame
    for index, row in df.iterrows():
        floors = row['Floors']
        street = row['Street']
        build_year = row['Build_year']

        build_string = (str(floors)+str(street)).strip().strip()

        for item in add_build_year:
            floors_street_year = item.strip()
            
            if floors_street_year.startswith(build_string):
                year = floors_street_year[-4:]
                year = re.findall(r'\d+', year)
                year = ''.join(year)
                df.at[index,'Build_year'] = year
                break

    return df

In [11]:
def find_avg_building_old_by_street():
    df = pd.read_csv("Data/Nadlan_clean.csv",index_col=0)
    df = df.dropna(subset=['Build_year','Street']).reset_index(drop=True)
    df['Build_year'] = df['Build_year'].astype(int)
    df['Street'] = df['Street'].astype(str)
    
    # create unique streets list
    streets = df['Street'].unique()
    
    street_buildings_avg = {}
    for street in streets:
        
        street_df = df[df['Street'] == street]
        street_mean = street_df['Build_year'].mean()
        
        
        street_buildings_avg[street] = street_mean
        
    return street_buildings_avg
        
def complete_building_old_by_street(df,avg_bulding_old_street):
    for index, row in df.iterrows():
        old = row['Build_year']
        street = row['Street']
        if old == 0:
            try:
                avg = avg_bulding_old_street[street] 
                df.at[index, 'Build_year'] = avg
            except:
                df.drop(index = index , inplace =True)
            
            
        
    df['Build_year'] = df['Build_year'].astype(np.int32)

    return df

df = complete_build_year_column(df,create_building_year_list())
df = complete_building_old_by_street(df,find_avg_building_old_by_street())

In [12]:
def calc_distance_from_the_see_TLV(X_coordinate, Y_coordinate):
    north_x = 180471
    north_y = 672391
    south_x = 177333
    south_y = 663016
    
    m = (south_y - north_y) / (south_x - north_x)   
    b = north_y - (m * north_x)
    
    numerator = abs(m * X_coordinate - Y_coordinate + b)
    denominator = math.sqrt(m**2 + 1)
    return numerator / denominator


def calc_distance_from_train_station(df):
    stations = [(179820.47,662424.54), (180619,664469.56), (181101.44,665688.78), (181710.96,667877.05)]
    distances = []

     # Iterate over each row in the DataFrame
    for index, row in df.iterrows():
        apartment_coords = (row['Lat'], row['Long'])

        # Calculate the distance between the apartment and each train station
        station_distances = []
        for station_coords in stations:       
            station_distance = abs(station_coords[0] - apartment_coords[1]) + abs(station_coords[1] - apartment_coords[0])
            station_distances.append(station_distance)

        # Find the minimum distance
        min_distance = min(station_distances)
        distances.append(min_distance)

    df['Train'] = distances
    df['Train'] = df['Train'].astype(np.int32)
    return df
    
def distance_from_sea_tlv(df):
    '''
    Need to fix long ? lat ? 
    '''
    df = df.dropna(subset=['Lat', 'Long']).reset_index(drop=True)
    latitudes = df['Long'].astype(float)
    longitudes = df['Lat'].astype(float)
    distances = [calc_distance_from_the_see_TLV(lat, lon) for lat, lon in zip(latitudes, longitudes)]
    df['Distance_sea'] = distances
    df['Distance_sea'] = df['Distance_sea'].astype(np.int32)
    return df

In [13]:
def edit_parking(df):
    # Not in use
    df['Parking'] = df['Parking'].str.replace('ללא','0')
    df['Parking'] = df['Parking'].astype(np.int32)
    return df

    
def add_date(df):
    current_year = datetime.now().year
    df['Year'] = current_year
    df.assign(profit='NAN')
    df['Year'].astype(np.int32)
    return df

def convert_price(df):
    df['Price'] = df['Price'].str.replace('לא צוין מחיר', '0')
    df['Price'] = df['Price'].str.replace('₪', '')
    df['Price'] = df['Price'].str.replace(',', '')
    df['Price'] = df['Price'].astype(np.int32)

    return df



def street_and_neighborhood_rank(df, column):
    df = update_neighborhood_street(df)
    df = df.dropna(subset=['Neighborhood','Street']).reset_index(drop=True)
    df_nadlan = pd.read_csv('Data/Nadlan_clean.csv', index_col=0)

        
    streets_rank = {}
    years = df_nadlan['Year'].unique()

    for year in years:
        df_by_year = df_nadlan[df_nadlan['Year'] == year]
        streets_rank[year] = {}

        for item in df_by_year[column].unique():
            df_by_item = df_by_year[df_by_year[column] == item]  # Filter df by item
            
            item_year_rank = np.nan
            if df_by_item['Size'].sum() != 0:
                item_year_rank = df_by_item['Price'].sum() / df_by_item['Size'].sum()

            streets_rank[year][item] = item_year_rank

    new_column_name = column + '_rank'
    df[new_column_name] = np.nan

    for index, row in df.iterrows():
        year = row['Year']
        item = row[column]
        found_street = False

        # Check for exact matches
        for nadlan_street in df_nadlan[column].unique():
            if item == nadlan_street:
                found_street = True
                item = nadlan_street
                break

        # Check for partial matches
        if not found_street and  found_street != False:
            for nadlan_street in df_nadlan[column].unique():
                if item in nadlan_street:
                    item = nadlan_street
                    found_street = True
                    break
                    
                check_parts = nadlan_street.split(' ')
                print(check_parts)
                if item in check_parts:
                    item = nadlan_street
                    found_street = True
                    break

        if found_street:
            for potential_year in [year, year-1, year-2 , year-3]:
                try:
                    df.at[index, new_column_name] = streets_rank[potential_year][item]
                    break
                except KeyError:
                    continue               
        else:
            pass
#             print(f"Could not find a match for {column} '{item}'")
            
    mean = df[new_column_name].mean()
    df[new_column_name] = df[new_column_name].fillna(mean)
    df[new_column_name] = df[new_column_name].astype(np.int32)
    return df



In [14]:
df = convert_price(df)
# df = distance_from_sea_tlv(df)
# df = calc_distance_from_train_station(df)
df = add_date(df)
df = street_and_neighborhood_rank(df,'Neighborhood')
df = street_and_neighborhood_rank(df,'Street')
df = street_and_neighborhood_rank(df,'Gush')


In [15]:
def check_for_match(df_nadlan, helka, year):
    helka = int(helka)
    helka_offsets = [helka - 1, helka + 1, helka + 2, helka - 2, helka + 3, helka - 3, helka + 4, helka - 4]
    for helka_offset in helka_offsets:
        for year_offset in range(4, 0, -1):
            year_temp = year - year_offset
            match = df_nadlan.loc[(df_nadlan['Gush_Helka'] == str(helka_offset)) & (df_nadlan['Year'] == year_temp), 'Helka_rank']
            if not match.empty:
                return match
    return match


def helka_rank(df):
    count = 0
    df = df.dropna(subset=['Gush','Helka']).reset_index(drop=True)
    df_nadlan = pd.read_csv('Data/Nadlan_clean.csv', index_col=0)
    df_nadlan['Gush_Helka'] = df_nadlan['Gush'].astype(str) + df_nadlan['Helka'].astype(str)
    
    column = 'Gush_Helka'

    df[column] = df['Gush'].astype(str) + df['Helka'].astype(str)
    df[column] = df[column].str.replace('.','')
    
    for index, row in df.iterrows():
        gush_helka = row['Gush_Helka']
        current_year = datetime.now().year
        year = current_year
        helka_rank = np.nan
        
        match = df_nadlan.loc[(df_nadlan['Gush_Helka'] == gush_helka) & df_nadlan['Year'] == year  ,'Helka_rank']
        
        if match.empty:
            match = check_for_match(df_nadlan, gush_helka , year)
    
        
        if not match.empty:
            df.at[index, 'Helka_rank'] = match.values[0]
        
        if match.empty:
            count+=1
    mean = df['Helka_rank'].mean()
    df['Helka_rank'] = df['Helka_rank'].fillna(mean)
    df['Helka_rank'] = df['Helka_rank'].astype(np.int32)
    print(count)
    return df



def check_for_match_update(df_nadlan, helka):
    helka = int(helka)
    helka_offsets = [helka - 1, helka + 1, helka + 2, helka - 2, helka + 3, helka - 3, helka + 4, helka - 4]
    for helka_offset in helka_offsets:
        match = df_nadlan.loc[(df_nadlan['Gush_Helka'] == str(helka_offset)), 'Helka_rank']
        if not match.empty:
            return match
    return match

def helka_rank_update(df):
    count = 0
    df = df.dropna(subset=['Gush','Helka']).reset_index(drop=True)
    df_nadlan = pd.read_csv('Data/Nadlan_clean.csv', index_col=0)
    df_nadlan['Gush_Helka'] = df_nadlan['Gush'].astype(str) + df_nadlan['Helka'].astype(str)
    
    column = 'Gush_Helka'

    df[column] = df['Gush'].astype(str) + df['Helka'].astype(str)
    df[column] = df[column].str.replace('.', '', regex=False)
    
    for index, row in df.iterrows():
        gush_helka = row['Gush_Helka']
        helka_rank = np.nan
        
        match = df_nadlan.loc[(df_nadlan['Gush_Helka'] == gush_helka),'Helka_rank']
        
        if match.empty:
            match = check_for_match_update(df_nadlan, gush_helka)
    
        if not match.empty:
            df.at[index, 'Helka_rank'] = match.values[0]
        
        if match.empty:
            count+=1
      
    mean = df['Helka_rank'].mean()
    df['Helka_rank'] = df['Helka_rank'].fillna(mean)
    df['Helka_rank'] = df['Helka_rank'].astype(np.int32)
    print(count)
    return df

In [16]:
df = helka_rank_update(df)
df.to_csv('Data/Yad2_clean.csv',index=False) 

57


In [17]:
def append_to_file():
    file_path = "status.txt"
    message = "yad_2_clean pass-success"

    try:
        # Open the file in append mode
        with open(file_path, "a") as file:
            # Append the message on a new line
            file.write("\n" + message)

        print("Message appended to the file successfully!")
    except Exception as e:
        print("An error occurred while appending to the file:", str(e))

# Call the function to append to the file
append_to_file()

Message appended to the file successfully!


In [18]:
df.shape

(1012, 30)